# 3D Scalar Advection Solver on a Sphere Using Radial Basis Functions

Samm Elliott

### The PDE
The transport equation for a passive tracer with mixing ratio $q$, without sources or sinks, can be written in the conservative flux form given by

$$\frac{\partial(\rho q)}{\partial t} + \nabla\cdot(\rho q \bar{u}) = 0.$$
$$\frac{\partial{q}}{\partial t} = - \bar{u}\cdot\nabla{q} .$$

where $\rho$ is the air density, $\bar{u}$ is the wind field and $\nabla$ is the standard 3D gradient operator. The mass continuity equation is given by

$$\frac{\partial\rho}{\partial t} + \nabla\cdot(\rho \bar{u}) = 0.$$

### Import Libraries

In [ ]:
import numpy as np
from numpy import linalg as nla
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d.axes3d import Axes3D
import scipy.sparse as ssparse
from scipy.sparse import linalg as sla
import time
import netCDF4 as nc4
import os

# Physical Constants

In [ ]:
a = 6.37122*(10**6)  # radius of the Earth (m)
g = 9.80616          # gravitational constant (m s^(-2))
p0 = 100000.0        # reference surface pressure (Pa)
cp = 1004.5          # specific heat capacity of dry air (J kg^(-1) K^(-1))
Rd = 287.0           # gas constant for dry air (J kg^(-1) K^(-1))
kappa = Rd/cp        # ratio of Rd to cp
T0 = 300.0           # isothermal atmosphere temperature (K)
pi = np.pi           # pi

### Simulation Parameterizations

In [ ]:
htop = 12000.0       # height position of model top (m)
Nh = 4096            # number of horizontal nodes
# Nh = 12100            # number of horizontal nodes
# Nv = 6              # number of vertical nodes
Nv = 24              # number of vertical nodes
n = 55               # RBF stencil size
dt = 1800

### Initializing the Nodeset

We first read in the specified maximal determinant (MD) nodes and add vertical levels for our nodeset. These MD nodes were aquired from http://web.maths.unsw.edu.au/~rsw/Sphere/. Check ./md directory for available nodesets.

In [ ]:
# read in MD nodes - note these are for a unit sphere
X_hat = np.loadtxt("nodesets/md."+str(Nh).zfill(5),usecols = (0,1,2)).T

If $\hat{x}$ is a MD nodepoint on a unit sphere, then the corresponding point $\bar{x}$ in our domain for vertical level $n$ is given by

$$\bar{x} = (a+n\frac{z_{top}}{N_v})\hat{x}$$

In [ ]:
# create nodeset
h = htop/(Nv-1)
rlvls = a + np.linspace(-3*h, htop + 3*h, num = Nv + 6)
X = np.tensordot(X_hat, rlvls, 0)

The following plots the MD nodes and the nodeset for our domain.

In [ ]:
# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(md_nodes[0],md_nodes[1],md_nodes[2])

In [ ]:
# fig = plt.figure(figsize=(20,20))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(xx[0],xx[1],xx[2])

### Creating n-point Stencils

We can just use the MD nodeset since the stencils will be valid for any radially scaled version of the nodeset. 

In [ ]:
%run RBFFD_Generation.ipynb
%time DM1h3,DM1v3,Lh,idx = get_RBFFD_DMs(X_hat,n)
# gamma = 145*Nh**-4
depth = 3

In [ ]:
# fig = plt.figure(figsize=(12,12))
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.scatter(X_hat[0],X_hat[1],X_hat[2],marker='o',c='blue',s=10)
# ax.scatter(X_hat[0,idx[0]],X_hat[1,idx[0]],X_hat[2,idx[0]],marker='o',c='green',s=50)
# ax.view_init(60,30)


## Test Case 2: Hadley-like Circulation

In [ ]:
%run TC1_3D_deformational_flow.ipynb
TC = 1
plttype = 0

In [ ]:
# %run TC2_Hadley_circulation.ipynb
# TC = 2
# plttype = 1

In [ ]:
r,lmbda,phi = c2s(X[0],X[1],X[2])

## Timestepping

In [ ]:
def set_ghosts(var,method):
    
    for i in range(1,depth+1):
        
        if method == 0:
            var[:,depth - i] = 0.0
            var[:,(Nv + depth - 1) + i] = 0.0
            
        if method == 1:
            var[:,depth - i] = var[:,depth + i]
            var[:,(Nv + depth - 1) + i] = var[:,(Nv + depth - 1) - i]
        
        if method == 2:
            var[:,depth - i] = 2*var[:,depth] - var[:,depth + i]
            var[:,(Nv + depth - 1) + i] = 2*var[:,Nv + depth - 1] - var[:,(Nv + depth - 1) - i]
            
    return var

In [ ]:
def eval_hyperviscosity(f,L,idx):
    
    # initialize gradient component arrays
    Lf = np.empty((Nh,Nv))
    
    ### evaluate horizontal components
    for i in range(Nh):
        
        L_i = L[i]
        
        for j in range(depth,Nv+depth):
            
            f_nbrs = f[idx[i],j]

            Lf[i,j-3] = np.vdot(f_nbrs,L_i)

    return Lf

def eval_gradient(f,DM1h3,DM1v3,idx):
    
    # horizontal/vertical differentiation scaling factors
    c_h = 1/rlvls
    c_v = 1/h
    
    # initialize gradient component arrays
    df_dx = np.empty((Nh,Nv))
    df_dy = np.empty((Nh,Nv))
    df_dz = np.empty((Nh,Nv))
    
    ### evaluate horizontal components
    for i in range(Nh):
        
        Dx_i = DM1h3[0,i]
        Dy_i = DM1h3[1,i]
        Dz_i = DM1h3[2,i]
        
        for j in range(depth,Nv+depth):
            
            f_nbrs = f[idx[i],j]
            
            df_dx[i,j-3] = np.vdot(f_nbrs,Dx_i)*c_h[j]
            df_dy[i,j-3] = np.vdot(f_nbrs,Dy_i)*c_h[j]
            df_dz[i,j-3] = np.vdot(f_nbrs,Dz_i)*c_h[j]
    
    ### evaluate vertical components
    for i in range(Nh):
        
        Dx_i = DM1v3[0,i]
        Dy_i = DM1v3[1,i]
        Dz_i = DM1v3[2,i]
        
        for j in range(depth,Nv+depth):
            
            f_nbrs = f[i,j-3:j+4]
            
            df_dx[i,j-3] += np.vdot(f_nbrs,Dx_i)*c_v
            df_dy[i,j-3] += np.vdot(f_nbrs,Dy_i)*c_v
            df_dz[i,j-3] += np.vdot(f_nbrs,Dz_i)*c_v
    
    return df_dx,df_dy,df_dz

def eval_RHS(q,U,DM1h3,DM1v3,Lh,idx):

    q = set_ghosts(q,1)
    
    dq_dx,dq_dy,dq_dz = eval_gradient(q,DM1h3,DM1v3,idx)
    
    Lq = eval_hyperviscosity(q,Lh,idx)
    
    gamma = -(1.5e4/a)*Nh**-4
    
    RHS = - ((U[0]*dq_dx) + (U[1]*dq_dy) + (U[2]*dq_dz)) + gamma*Lq
    
    return RHS


In [ ]:
##### 1st order DM tests

# ### Test 1
# z = (r-a)
# ff = (z/htop)**2
# ff_x1 = ((2*X[0]*z)/(r*(htop**2)))[:,depth:Nv+depth]
# ff_y1 = ((2*X[1]*z)/(r*(htop**2)))[:,depth:Nv+depth]
# ff_z1 = ((2*X[2]*z)/(r*(htop**2)))[:,depth:Nv+depth]

# ### Test 2
# ff = np.sin(X[0]/a + pi/4) * np.sin(X[1]/a + pi/4) * np.sin(X[2]/a + pi/4) * a
# ff_x1 = ((np.cos(X[0]/a + pi/4) * np.sin(X[1]/a + pi/4) * np.sin(X[2]/a + pi/4)))[:,depth:Nv+depth]
# ff_y1 = ((np.sin(X[0]/a + pi/4) * np.cos(X[1]/a + pi/4) * np.sin(X[2]/a + pi/4)))[:,depth:Nv+depth]
# ff_z1 = ((np.sin(X[0]/a + pi/4) * np.sin(X[1]/a + pi/4) * np.cos(X[2]/a + pi/4)))[:,depth:Nv+depth]


# ### Evaluate Diffs
# print("Testing DMs:")
# ff_x2 = rbffd_diff_h(ff,DM1h3[0],idx,1) + rbffd_diff_v(ff,DM1v3[0],1)
# ff_y2 = rbffd_diff_h(ff,DM1h3[1],idx,1) + rbffd_diff_v(ff,DM1v3[1],1)
# ff_z2 = rbffd_diff_h(ff,DM1h3[2],idx,1) + rbffd_diff_v(ff,DM1v3[2],1)
# print("\tf:","\n\t\tmin = {0:e}".format(np.min(ff)),"\n\t\tmax = {0:e}".format(np.max(ff)))
# print("\tf_x:","\n\t\tmax(f_x) = {0:e}".format(np.max(np.abs(ff_x1))),"\n\t\tmax_err(f_x) = {0:e}".format(np.max(np.abs(ff_x1-ff_x2))),"\n\t\tmedian_err(f_x) = {0:e}".format(np.median(np.abs(ff_x1-ff_x2))),"\n\t\tave_err(f_x) = {0:e}".format(np.mean(np.abs(ff_x1-ff_x2))))
# print("\tf_y:","\n\t\tmax(f_y) = {0:e}".format(np.max(np.abs(ff_y1))),"\n\t\tmax_err(f_y) = {0:e}".format(np.max(np.abs(ff_y1-ff_y2))),"\n\t\tmedian_err(f_y) = {0:e}".format(np.median(np.abs(ff_y1-ff_y2))),"\n\t\tave_err(f_y) = {0:e}".format(np.mean(np.abs(ff_y1-ff_y2))))
# print("\tf_z:","\n\t\tmax(f_z) = {0:e}".format(np.max(np.abs(ff_z1))),"\n\t\tmax_err(f_z) = {0:e}".format(np.max(np.abs(ff_z1-ff_z2))),"\n\t\tmedian_err(f_z) = {0:e}".format(np.median(np.abs(ff_z1-ff_z2))),"\n\t\tave_err(f_z) = {0:e}".format(np.mean(np.abs(ff_z1-ff_z2))))


In [ ]:
def plot_var(var):
    
    plt.ion()
    
    if plttype == 0:
        fig = plt.figure(figsize=(10,6))
        ax = fig.add_subplot(1,1,1)
        pltlvl = int(np.floor(5000/(2*h))+3)
        cax = ax.scatter(lmbda[:,0],phi[:,0],c = var[:,pltlvl])
        cbar = fig.colorbar(cax)
        
    if plttype == 1:
        fig = plt.figure(figsize=(16,6))
        ax = fig.add_subplot(1,1,1)
        cond = np.abs(lmbda-pi) < .1
        cax = ax.scatter(np.extract(cond,phi),np.extract(cond,r)-a,c = np.extract(cond,var),s=1.5e3/Nv)
        cbar = fig.colorbar(cax)
        
    plt.show()

In [ ]:
nsteps = int(tau/dt)
plot_int = 24

q_init = set_ghosts(get_q_init(r,lmbda,phi),0)
q = np.copy(q_init)
q_temp = np.copy(q_init)

hist_int = 1
q_hist = q_init[:,3:Nv+3].reshape((1,Nh,Nv))
hist_step = 0

for tstep in range(nsteps):
    
    start = time.time()
        
    # RK4 step 1
    t = tstep*dt
    U = u_t(r,lmbda,phi,t)[:,:,depth:Nv+depth]
    RHS = eval_RHS(q,U,DM1h3,DM1v3,Lh,idx)
    d = RHS
    
    # RK4 step 2
    t += dt/2
    U = u_t(r,lmbda,phi,t)[:,:,depth:Nv+depth]
    q_temp[:,depth:Nv+depth] = q[:,depth:Nv+depth] + (dt/2)*RHS
    RHS = eval_RHS(q_temp,U,DM1h3,DM1v3,Lh,idx)
    d += 2*RHS
    
    # RK4 step 3
    q_temp[:,depth:Nv+depth] = q[:,depth:Nv+depth] + (dt/2)*RHS
    RHS = eval_RHS(q_temp,U,DM1h3,DM1v3,Lh,idx)
    d += 2*RHS
    
    # RK4 step 4
    t += dt/2
    U = u_t(r,lmbda,phi,t)[:,:,depth:Nv+depth]
    q_temp[:,depth:Nv+depth] = q[:,depth:Nv+depth] + dt*RHS
    RHS = eval_RHS(q_temp,U,DM1h3,DM1v3,Lh,idx)
    d += RHS
    
    # Update tracers
    q[:,depth:Nv+depth] += (dt/6)*d
    
    t_total = time.time() - start
    
    
    print("Finished time step",tstep+1,"of",nsteps,"total -> {0:.1f} seconds/timestep".format(t_total))
    
    if (tstep+1)%plot_int == 0:
        plot_var(q)
    
    if (tstep+1)%hist_int == 0:
        hist_step += 1
        q_hist = np.append(q_hist,q[:,3:Nv+3].reshape((1,Nh,Nv)),axis=0)
        

In [ ]:
# outputFile = "results/TC"+str(TC)+"_"+str(Nh)+"h_"+str(Nv)+"v.nc"
# os.remove(outputFile)
# ncf_root = nc4.Dataset(outputFile,"w","NETCDF4")
# ncg_sol = ncf_root.createGroup("solution")
# ncd_time = ncf_root.createDimension("time", hist_step+1)
# ncd_hid = ncf_root.createDimension("hid", Nh)
# ncd_vid = ncf_root.createDimension("vid", Nv)
# ncv_q = ncg_sol.createVariable("q","f8",("time","hid","vid"))
# ncv_q = q_hist
# ncf_root.close()

In [ ]:
### Extrema
q_min = np.min(q)
q_max = np.max(q)

### Solution Error Norms
l1 = np.sum(np.abs(q_init - q)) / np.sum(np.abs(q_init))
l2 = np.sqrt(np.sum((q_init - q)**2) / np.sum(q_init**2))
linf = np.max(np.abs(q_init - q)) / np.max(np.abs(q_init))

### Mass Conservation
rho_0 = rho(r)
V = (4*pi*r**2)*h/Nh
Mt_tot = np.sum(rho_0*V*q)
M0_tot = np.sum(rho_0*V*q_init)

### Print Error Results
print("Solution Extrema:\n\tMin:  \t{0:.2e}".format(q_min),"\n\tMax:  \t{0:.2e}".format(q_max))
print("Error Results:\n\tL_1:  \t{0:.2e}".format(l1),"\n\tL_2:  \t{0:.2e}".format(l2),"\n\tL_inf:\t{0:.2e}".format(linf))
print("\nTracer Mass Difference:\t%{0:.3f}".format(np.abs(100*(Mt_tot-M0_tot)/M0_tot)))

# ## Plot Final Solution
# pltlvl = int(np.floor((z_1+z_2)/(2*h))+depth)
# fig = plt.figure(figsize=(16,8))
# ax = fig.add_subplot(1,1,1)
# cax = ax.scatter(lmbda[:,0],phi[:,0],c = q[:,pltlvl])
# cbar = fig.colorbar(cax)


In [ ]:
# Config: Nh = 4096, Nv = 30, n = 55, dt = 1800, 
# Extrema: Min = -.21, Max = 1.08 --- at tau/2: q_min = -0.72, q_max = 1.88
# Error: L_1 = .083, L_2 = .077, L_inf = .26
# Tracer Mass Difference: %.87
# Timing: 10.5 seconds/timestep

# Config: Nh = 12100, Nv = 30, n = 55, dt = 1800, 
# Extrema: Min = -.056, Max = .96 
# Error: L_1 = .037, L_2 = .033, L_inf = .093
# Tracer Mass Difference: %0.005
# Timing: 27 seconds/timestep


In [ ]:
# for i in range(Nv+6):
#     print("h_lvl = {0:2d}".format(i),", {0:6d}m".format(int(h*(i-3))),"\tq_min = {0:4.2f}".format(np.min(q_init[:,i])),",{0:4.2f}".format(np.min(q[:,i])),"\tq_max = {0:4.2f}".format(np.max(q_init[:,i])),",{0:4.2f}".format(np.max(q[:,i])))

In [ ]:
# ### Plot Final Solution
# fig = plt.figure(figsize=(16,8))
# ax = fig.add_subplot(1,1,1)
# cond = np.abs(lmbda-pi) < .1
# cax = ax.scatter(np.extract(cond,phi),np.extract(cond,r)-a,c = np.extract(cond,q),s=100)
# cbar = fig.colorbar(cax)